In [2]:
import os
os.chdir('..')

In [15]:
import pysindy as ps
import numpy as np
import matplotlib.pyplot as plt

import Yukawa_SINDy as ys
import cross_validation as cv

from importlib import reload
import pickle as pkl
with open('scaling_const.float', 'rb') as f:
    SCALING_CONST = pkl.load(f)

In [26]:
reload(ys)
reload(cv)

<module 'cross_validation' from '/Users/zbh0005/Library/CloudStorage/OneDrive-AuburnUniversity/Documents/Code/yukawa-sindy/cross_validation.py'>

# Comparison of strong- and weak-form SINDy on representative cases

## Low noise

For this case, we will utilize a noise standard deviation of $1\times10^{-4}$ to demonstrate that weak-form SINDy performs essentially the same as the strong-form.

In [28]:
# generate data
noise_level = 1e-4

simulation_list = ys.generate_training_data(
    n_sims = 200,
    duration = 5,
    dt = 0.001,
    noise_level = noise_level,
    mu_x0s = 0.5,
    mu_v0s = 0.01,
    scaled = True
)

In [29]:
norm_mat = np.array([1, SCALING_CONST]).reshape(2,1).repeat(10, axis=1)
true_coefficients = np.array(
    [[0., 1., 0.,            0., 0.,            0., 0., 0., 0., 0.],
     [0., 0., SCALING_CONST, 0., SCALING_CONST, 0., 0., 0., 0., 0.]]
)

def scan_thresholds(threshold_space, simulation_list, feature_library, feature_names, true_coefficients, n_folds):
    all_best_models = []
    all_best_model_scores = []
    all_truest_models = []
    all_truest_model_scores = []
    for threshold in threshold_space:
        best_model, best_model_score, truest_model, truest_model_score = cv.cross_validate(
            simulation_list,
            threshold,
            feature_library,
            feature_names,
            true_coefficients,
            n_folds
        )
        all_best_models.append(best_model)
        all_best_model_scores.append(best_model_score)
        all_truest_models.append(truest_model)
        all_truest_model_scores.append(truest_model_score)
    return all_best_models, all_best_model_scores, all_truest_models, all_truest_model_scores

### Strong library

In [30]:
threshold_space = np.linspace(0, 0.3, 10)
feature_library = ys.generate_Yukawa_library()
feature_names = ['x', 'v']
n_folds = 10

# threshold analysis
model_info = scan_thresholds(threshold_space, simulation_list, feature_library, feature_names, true_coefficients, n_folds)
# unpack
all_best_models, all_best_model_scores, all_truest_models, all_truest_model_scores = model_info

/opt/anaconda3/envs/yukawa-sindy-vetted/lib/python3.12/site-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.03333333333333333) and eliminated all coefficients
  warnings.warn(
/opt/anaconda3/envs/yukawa-sindy-vetted/lib/python3.12/site-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.06666666666666667) and eliminated all coefficients
  warnings.warn(
/opt/anaconda3/envs/yukawa-sindy-vetted/lib/python3.12/site-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/opt/anaconda3/envs/yukawa-sindy-vetted/lib/python3.12/site-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.13333333333333333) and eliminated all coefficients
  warnings.warn(
/opt/anaconda3/envs/yukawa-sindy-vetted/lib/python3.12/site-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.1666

In [ ]:
directory_name = 'paper_results/weak_vs_strong/'
file_name = 'strong_models_and_scores.list'
def pickle_data(data, directory_name:str, file_name:str, overwrite:bool = False):
    file_path = directory_name + file_name
    if os.exists(file_path) and not overwrite:
        raise Exception('File exists and overwrite is set to False')
    if not os.exists(directory_name):
        os.mkdir(directory_name)
    if not os.exists(file_path):
        with open(file_path, 'wb') as f:
            pkl.dump(data, f)


In [31]:
for idx in range(len(all_best_models)):
    print('threshold:', threshold_space[idx])
    print('truest')
    all_truest_models[idx].print()
    print('score', all_truest_model_scores[idx])

    print('\nbest prediction')
    all_best_models[idx].print()
    print('score', all_best_model_scores[idx])
    print(60*'-')


threshold: 0.0
truest
(x)' = 1.000 v + 0.007 exp(-x) / x + -0.005 exp(-x) / x^2 + 0.001 exp(-x) / x^3
(v)' = -0.003 x + 0.011 v + 3.458 exp(-x) / x + 5.254 exp(-x) / x^2 + -0.109 exp(-x) / x^3 + 0.004 exp(-x) / x^4
score 0.08258992957367231

best prediction
(x)' = 1.000 v + 0.007 exp(-x) / x + -0.005 exp(-x) / x^2 + 0.001 exp(-x) / x^3
(v)' = -0.003 x + 0.011 v + 3.458 exp(-x) / x + 5.254 exp(-x) / x^2 + -0.109 exp(-x) / x^3 + 0.004 exp(-x) / x^4
score 0.08258992957367231
------------------------------------------------------------
threshold: 0.03333333333333333
truest
(x)' = 1.000 v
(v)' = 4.547 exp(-x) / x + 4.545 exp(-x) / x^2
score 0.0714912949336797

best prediction
(x)' = 1.000 v
(v)' = 4.547 exp(-x) / x + 4.545 exp(-x) / x^2
score 0.0714912949336797
------------------------------------------------------------
threshold: 0.06666666666666667
truest
(x)' = 1.000 v
(v)' = 4.547 exp(-x) / x + 4.545 exp(-x) / x^2
score 0.0714912949336797

best prediction
(x)' = 1.000 v
(v)' = 4.547 ex

### Weak case

In [40]:
feature_library = ys.generate_weak_Yukawa_library(
    simulation_list[0].t,
    K = 500
)
# threshold analysis
model_info = scan_thresholds(threshold_space, simulation_list, feature_library, feature_names, true_coefficients, n_folds)
# unpack
all_best_weak, all_best_scores_weak, all_truest_weak, all_truest_scores_weak = model_info

/opt/anaconda3/envs/yukawa-sindy-vetted/lib/python3.12/site-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.03333333333333333) and eliminated all coefficients
  warnings.warn(
/opt/anaconda3/envs/yukawa-sindy-vetted/lib/python3.12/site-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.06666666666666667) and eliminated all coefficients
  warnings.warn(
/opt/anaconda3/envs/yukawa-sindy-vetted/lib/python3.12/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/opt/anaconda3/envs/yukawa-sindy-vetted/lib/python3.12/site-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/opt/anaconda3/envs/yukawa-sindy-vetted/lib/python3.12/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.13333333333333333) a

## High noise case